Universidad del Valle de Guatemala <br>
Security Data Science

# LABORATORIO 3
## Clasificación de Malware

Walter Saldaña #19897
___

importar librerias

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score 
from sklearn.metrics import accuracy_score, precision_score, recall_score
from xgboost import XGBClassifier

/home/walt/.local/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


cargar datos

In [2]:
df = pd.read_csv('VirusSample.csv')

In [3]:
df.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus


# Análisis exploratorio

Estadística descriptiva

In [4]:
df.describe()

,file,api,class
count,9795,9795,9795
unique,9795,2980,13
top,7ff49f2f0912352416b05c010f35f402cc79feed,"LoadLibraryA,GetProcAddress",Trojan
freq,1,1007,6153


Tipos de datos dentro del dataset

In [5]:
df.dtypes

file     object
api      object
class    object
dtype: object

Distintas clases que contiene el dataset

In [6]:
df['class'].unique()

array(['Virus', 'Riskware', 'Spyware', 'Downloader', 'Dropper', 'Agent',
       'Adware', 'Trojan', 'Backdoor', 'Worms', 'Ransomware', 'Keylogger',
       'Crypt'], dtype=object)

Distribución de las clases

In [7]:
df.groupby('class')['file'].count().sort_values()

class
Keylogger        1
Crypt            2
Dropper          4
Riskware         4
Ransomware      10
Spyware         11
Downloader      31
Agent          102
Adware         222
Worms          441
Backdoor       447
Virus         2367
Trojan        6153
Name: file, dtype: int64

Búsqueda de datos incompletos o inválidos

In [8]:
df.isnull().sum()

file     0
api      0
class    0
dtype: int64

In [9]:
df.isna().sum()

file     0
api      0
class    0
dtype: int64

# Preprocesamiento

Convertir a lista el string que contiene las apis del virus para poder iterarlas

In [10]:
df['api'] = df['api'].str.split(',')
df.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"[IntersectRect, GetCurrentProcess, GetVersion]",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"[GetCaretBlinkTime, CountClipboardFormats, Get...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"[VarR8Pow, GetClipboardViewer, GetInputDesktop...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"[SetTraceCallback, CopyAcceleratorTableW, GetP...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"[SHLoadNonloadedIconOverlayIdentifiers, VarUI8...",Virus


Realizar un label encoder de la clase de virus para optimizar el entrenamiento del modelo

In [11]:
le = preprocessing.LabelEncoder()
df['class'] = le.fit_transform(df['class'])
df.sample(10)

,file,api,class
8712,50f7c3b43941a2a03e5d1334382387939bb8523c,"[WSAGetLastError, WSAStartup, __WSAFDIsSet, ac...",2
5447,fc9553406ad6804d4fc77ce029892aa57be42a7e,[_CorExeMain],10
9256,8dfb9fa85f4ed562fa4dcb6464a6153b588a6000,"[RegCloseKey, LoadLibraryA, ExitProcess, GetPr...",12
3451,06d21e2b62fcc1bf3444bd5f42ff087f2ab26571,"[GetModuleBaseNameA, GetModuleFileNameExA, Enu...",1
6590,d5de7cb9e1079f5b20f589664a27713e9e939e7d,"[LoadLibraryA, GetProcAddress, VirtualProtect,...",10
2826,33cef0b2dd78eccc6a587202c34f9a7f528456ab,"[RtlLargeIntegerShiftRight, ZwReadFile, FreeCo...",11
2207,9dbe3ec10ab21ac69cd12b776b8b38d0ab75669e,"[NtCreateJobSet, GetOpenClipboardWindow, GetCu...",11
4463,e99b9dc167c07127043817a1029ec42e27034728,"[GetModuleHandleA, GetProcAddress, CryptHashDa...",10
5142,77d2382d9aa33d9ceb078b5dd3b5c817f6eec176,"[MessageBoxA, wsprintfA, ExitProcess, GetModul...",10
5366,9f866dfd4c0309a98639625273804a09d0621b9d,"[LoadLibraryA, GetProcAddress, VirtualProtect,...",10


# Ingeniería de características
El algoritmo dura bastante por lo que se guardo el resultado en un csv para extraerlo luego sin tener que volver a correrlo

Se crea una columna por cada api y se utiliza un valor binario para indicar si la fila contiene esa api. Para evitar errores de alocamiento de memoria, el algoritmo se trabaja por partes y se concatenan los resultados.

In [12]:
'''
dummies = pd.Series()
jump_value = 1000
max_value = df['api'].count()

for i in range(0, max_value, jump_value):
    nxt = i + jump_value
    limit = nxt if nxt < max_value else None
    dummies_ext = df.iloc[i:limit]['api'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()
    dummies = pd.concat([dummies, dummies_ext])
    print(f'Transformed {nxt} - {len(dummies_ext.index)}')
'''

"\ndummies = pd.Series()\njump_value = 1000\nmax_value = df['api'].count()\n\nfor i in range(0, max_value, jump_value):\n    nxt = i + jump_value\n    limit = nxt if nxt < max_value else None\n    dummies_ext = df.iloc[i:limit]['api'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()\n    dummies = pd.concat([dummies, dummies_ext])\n    print(f'Transformed {nxt} - {len(dummies_ext.index)}')\n"

Ya que se concatenan los resultados por partes, varios grupos de filas no tendrán las mismas columnas que las otras, por lo que al concatenarlas habrán bastantes NaN, pero sabemos que si tiene NaN es porque no tiene esa columnay por lo tanto no tiene esa API, así que se puede reemplazar por 0 para indicar que no la tiene.

In [13]:
# dummies = dummies.fillna(0)
# dummies

Para evitar correr el algoritmo otra vez, se guarda el resultado en un csv

In [14]:
# dummies.to_csv('dummies.csv')

Si el algoritmo anterior ya se ejecutó, se carga el dataset

In [15]:
dummies = pd.read_csv('dummies.csv')

In [16]:
col_names = dummies.columns
col_names

Index(['Unnamed: 0', '0', '??0?$basic_ios@DU?$char_traits@D@std@@@std@@IAE@XZ',
       '??0?$basic_ios@DU?$char_traits@D@std@@@std@@IEAA@XZ',
       '??0?$basic_istream@DU?$char_traits@D@std@@@std@@QEAA@PEAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z',
       '??0?$basic_ostream@DU?$char_traits@D@std@@@std@@QAE@PAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z',
       '??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IAE@XZ',
       '??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IEAA@XZ',
       '??0_Container_base12@std@@QAE@XZ', '??0_Locinfo@std@@QAE@HPBD@Z',
       ...
       'rtcIsArray', 'rtcIsMissing', 'rtcIsObject', 'rtcJoin', 'rtcLeftVar',
       'rtcPackTime', 'rtcRandomNext', 'rtcVarBstrFromAnsi',
       'rtcVarDateFromVar', 'wcscspn'],
      dtype='object', length=7967)

Se conservan únicamente las columnas necesarias para el modelo para evitar redundancia, sobrecarga y sesgos.

In [17]:
df = pd.concat([df, dummies], axis=1)
del df['Unnamed: 0']
del df['0']
del df['api']
del df['file']
df.head()

,class,??0?$basic_ios@DU?$char_traits@D@std@@@std@@IAE@XZ,??0?$basic_ios@DU?$char_traits@D@std@@@std@@IEAA@XZ,??0?$basic_istream@DU?$char_traits@D@std@@@std@@QEAA@PEAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z,??0?$basic_ostream@DU?$char_traits@D@std@@@std@@QAE@PAV?$basic_streambuf@DU?$char_traits@D@std@@@1@_N@Z,??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IAE@XZ,??0?$basic_streambuf@DU?$char_traits@D@std@@@std@@IEAA@XZ,??0_Container_base12@std@@QAE@XZ,??0_Locinfo@std@@QAE@HPBD@Z,??0_Lockit@std@@QAE@H@Z,...,rtcIsArray,rtcIsMissing,rtcIsObject,rtcJoin,rtcLeftVar,rtcPackTime,rtcRandomNext,rtcVarBstrFromAnsi,rtcVarDateFromVar,wcscspn
0,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Modelo 1: Support Vector Machine

## Implementación y validación

Split data

In [18]:
X = df.loc[:, df.columns != 'class']
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

Crear y entrenar el modelo

In [19]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

Validar predicciones

In [20]:
y_pred = clf.predict(X_test)

## Validación cruzada

In [21]:
# k_folds = KFold(n_splits = 10)
scores = cross_val_score(clf, X, y, cv = 10)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

/home/walt/.local/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Cross Validation Scores:  [0.94693878 0.95918367 0.93877551 0.93469388 0.93265306 0.93054137
 0.93564862 0.93769152 0.93667007 0.94484168]
Average CV Score:  0.9397638156386151
Number of CV Scores used in Average:  10


## Métricas

In [22]:
accuracy_score(y_test, y_pred)

0.9445389588295339

Lo que nos muestra el accuracy es que el modelo logró predecir correctamente un 94% de los datos de testing, lo cual es un valor bastante aceptable para la detección de malware.

In [23]:
precision_score(y_test, y_pred, average='weighted', zero_division=False)

0.9405462285134326

La precisión nos indica que el 94% de los datos catalogados como positivos en una clase, realmente pertenecían a esa categoría.

In [24]:
recall_score(y_test, y_pred, average='weighted', zero_division=False)

0.9445389588295339

Del recall se interpreta que solo el 94% de los datos que verdaderamente pertenecen a una clase, fueron correctamente identificados como tal.

# Modelo 2: Extreme Gradient Booster

## Implementación y validación

In [25]:
bst = XGBClassifier(n_estimators=10, max_depth=2, learning_rate=1, objective='hist')

In [26]:
bst.fit(X_train, y_train)

/home/walt/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/walt/.local/lib/python3.9/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[02:31:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [27]:
y_pred_xgb = bst.predict(X_test)

## Validación cruzada

In [28]:
k_folds = KFold(n_splits = 10)
scores = cross_val_score(bst, X, y, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

/home/walt/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/walt/.local/lib/python3.9/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[02:32:14] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/walt/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/walt/.local/lib/python3.9/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[02:32:34] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/walt/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/walt/.local/lib/python3.9/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


...

[02:34:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/walt/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/walt/.local/lib/python3.9/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[02:34:48] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/walt/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/walt/.local/lib/python3.9/site-packages/xgboost/data.py:208: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


[02:35:07] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross Validation Scores:  [0.88673469 0.73979592 0.00918367 0.75204082 0.9255102  0.85903984
 0.7875383  0.773238   0.92236977 0.00715015]
Average CV Score:  0.6662601363323674
Number of CV Scores used in Average:  10


## Métricas

In [29]:
accuracy_score(y_test, y_pred_xgb)

0.9118747873426335

Para el accuracy se observa que el 91% de los datos fueron clasificados correctamente a la clase que pertenece.

In [30]:
precision_score(y_test, y_pred_xgb, average='weighted', zero_division=False)

0.8970088120384435

Aquí el precision nos muestra que el 89% de los datos catalogados dentro de una clase, realmente pertenecen a esa clase.

In [31]:
recall_score(y_test, y_pred_xgb, average='weighted', zero_division=False)

0.9118747873426335

Y por último el recall muestra que el modelo fue capaz de identificar correctamente 91% de los datos pertenecientes a una clase.

___
# Conclusiones
**El artículo “New Datasets for Dinamyc Malware Classification” sirve como un benchmark para comparar modelos de clasificación, ¿se lograron obtener mejores métricas que las obtenidas en el artículo para la clasificación de malware?**

En el caso del SVM los resultados obtenidos son virtualmente los mismos a los presentados en el artículo, con un accuracy de 94%.
Para el EGB, los resultados fueron mínimamente mayores comparados a los del artículo, donde se ibtuvo accuracy del 90%, mientras que en este ejemplar se obtuvo del 91%. La diferencia radica en los parámetros del modelo, pues si bien ambos utilizamos el algoritmo arból de hist, en esta práctica se experimentó con el resto de valores, notando que al aumentar n_estimators a 10, los resultados mejoraban considerablemente.
Esta claro que también la forma de preprocesar los datos pudo haber inlfuido la respuesta, sin embargo el dataset utilizado ya se encontraba trabajado por lo que los cambios fueron mínimos.